In [2]:
import scraping_class
import requests
import random
import tqdm
import pandas as pd
import time
import numpy as np
import datetime
import pytest
import requests
from connector import Connector
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile) # for logging

from numpy.random import randint
import logging


#
from selenium import webdriver
import re
import os
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [3]:
# start chrome driver, has to be in folder.
dirname = os.getcwd()
path = os.path.join(dirname, 'chromedriver')
# log from Chrome
log_path = os.path.join(dirname, 'url_log.log')
driver = webdriver.Chrome(executable_path=path, service_args=["--verbose", f"--log-path={log_path}"])


In [6]:
''' 
Finds and extracts all urls from a given search term.
saves the urls every 100 loops, kan be interrupted at any time when enough urls have been collected.
*** Click ok on the cookie prompt ***
'''

search_term = 'a'
connector = Connector('log_urls.csv')
url = f'https://ekstrabladet.dk/find/?g=true&q={search_term}'
# to save log not used by selenium
response,call_id = connector.get(url, 'urls_get')
driver.get(url)
time.sleep(1)
# Next page button
next_page = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[4]/div[1]/div/div/div/div[32]/button')
# show all comment

loops = 0
url_pattern = 'https:\/\/[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}[-a-zA-Z0-9@:%_\+.~#?&//=;]*'
for i in range(3): # how many times to press next
    loops += 1
    try:
        next_page.click() # when this shows an error, i.e. no more comments, it breaks.
        time.sleep(0.5)
        if loops %100 == 0: # save every 100 loops
            # get soup
            search_soup = BeautifulSoup(driver.page_source, 'lxml')
            articles = search_soup.find_all(class_='flex-item mar-l--b width-1of1') # links soup
            print(f'{loops} out of 100,000')
            urls = []
            
            for i in range(len(articles)):
                urls.append(re.findall(url_pattern, str(articles[i]))) # save url
            
            print(f'there are now {len(urls)} urls')
            
            # save to cvs
            df_urls = pd.DataFrame(data=urls)
            df_urls.to_csv("urls.csv", index=False)
    except: # if there are no more articles or something breaks
        print('check the cookie prompte and try again') # stop :I
        break

In [8]:
# save soup
with open("search_html.txt", "w", encoding="utf-8") as f:
    f.write(search_soup.get_text())
    f.close()

In [3]:
# load urls
urls = pd.read_csv("urls.csv")

In [4]:
# remove plus articles
urls_no_plus = [ url for url in urls['0'] if "https://ekstrabladet.dk/plus" not in url ]

In [32]:
'''
The data is being split up into smaller chunks to scrape in on multiple computers.
this is done in the bots folder.
'''
#split into equal chunks of urls.
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
chunks = list(chunks(urls_no_plus, 8800))
print(f'there are {len(chunks)} chunks')
chunk0 = chunks[0]
chunk1 = chunks[1]
chunk2 = chunks[2]
chunk3 = chunks[3]
# save chunks
chunk0 = pd.DataFrame(data=chunk0)
chunk0.to_csv("chunk0.csv", index=False)
chunk1 = pd.DataFrame(data=chunk1)
chunk1.to_csv("chunk1.csv", index=False)
chunk2 = pd.DataFrame(data=chunk2)
chunk2.to_csv("chunk2.csv", index=False)
chunk3 = pd.DataFrame(data=chunk3)
chunk3.to_csv("chunk3.csv", index=False)

there are 4 chunks


In [ ]:
# load scraped data
df_scraped = pd.read_csv("data.csv")

In [36]:
'''
The following code takes the scraped headlines, articles, dates etc.
and converts it to a df and the dates to timeseries.
'''

# convert to df and drop nan
df = pd.DataFrame(df_scraped).dropna()
df=df.reset_index(drop=True)

x=len(df)
dato=[]

# convert times 
for i  in range(x):
    data_time_string=df[3][i]
    x=re.sub('\.',"",data_time_string)
    y=datetime.datetime.strptime(x, '%d %b %Y')
    dato.append(y)
    time.sleep(0.5)
    

df['time']=pd.Series(dato).values

In [ ]:
'''
This code is not used as it takes too much time and the dataset becomes way too big for our computers to handle.

Saves the comments in a list.
Does not save arcticle headline etc.
'''
# open comments
comments= []
i = 0 # to only run loop x times
for i, url in enumerate(urls):
    # initiate check variables
    i2 = 0
    i3 = 0
    no_comments = False
    
    i +=1
    print(f"i = {i}")
    if i == 50: # break after 10 articles
        break
    
    print(url[0])
    driver.get(url[0]) # open the current url
    time.sleep(5)
    '''
    Tries to find the show more comments button.
    break after 10 seconds of trying
    '''
    while True: # infinite loop
        try:
            # show comments button
            show_comments = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[4]/div[3]/div[2]/div')
            show_comments.click() # click it
            time.sleep(5)
            break
        except:
            time.sleep(1) # waits a total of 10 seconds i2*10 for the page to load
            i2 +=1
            print(f"i2 = {i2}")
            if i2 == 5:
                no_comments = True
                break
   
    if no_comments: # next article
        continue
    '''
    Tries to find the comment url.
    if the url is blank wait and try again.
    break after 10 seconds
    '''
    while True: # infinite loop
        try:
            show_comments_soup = BeautifulSoup(driver.page_source, 'lxml') # save soup
            comment_link = show_comments_soup.find_all(id='talkStream_iframe') # div with the comment link
            comment_url = re.findall(url_pattern, str(comment_link)) # save the comment url
            print(comment_url)
            if comment_url == []:
                i3 += 1
                print(f"i3 = {i3}")
                time.sleep(1)
                if i3 == 5:
                    break
                continue
            break
        except:
            break
            
    # open the comment_url to save comments, break if not possible
    while True: # infinite loop
        try:
            driver.get(comment_url[0])
            time.sleep(1)
            break
        except:
            break
    
    '''
    tries to show all comments.
    when there are no more next_comments it breaks, also breaks if the button does not exist
    '''
    while True: # infinite loop
        try:
            # more comments button
            next_comment = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div/div[3]/div[2]/div/div/div/div/div[3]/button')
            next_comment.click()
        except:
            time.sleep(1)
            break
    
    '''
    save the extracted comments to list.
    breaks if no comments were extracted
    '''
    while True:
        try:
            comment_soup = BeautifulSoup(driver.page_source, 'lxml')
            comments_ = comment_soup.find_all(class_="talk-plugin-rich-text-text CommentContent__content___ZGv1q")
            if comments_ == []:
                break
            else:
                comments.append(comments_)
                break
        except:
            break

In [46]:
# save the comments:
df_comments = pd.DataFrame(data=comments)
df_comments.to_csv("comments.csv", index=False)